In [25]:
import os
import pandas as pd
import re
import toml
import unicodedata

from datetime import datetime
from PyPDF2 import PdfReader

In [26]:
config_path = './config.toml'

with open(config_path, 'r') as f:

    cfg = toml.load(f)

def strip_accents(s):
   s=s.lower()
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def limpiar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = unicodedata.normalize('NFKD', texto)  # Normalizar texto
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])  # Eliminar tildes
    return texto

directory = './ModeloFacturas'

In [27]:
for filename in os.listdir(directory):

        if filename.endswith(".pdf"):

            filepath = os.path.join(directory, filename)                 

            print(f"Leyendo archivo: {filename}")

            #texto = leer_pdf(filepath)

            with open(filepath, 'rb'):
                
                lector_pdf = PdfReader(filepath)
                
                #Extraer solo las paginas 2 y 3.
                texto=lector_pdf.pages[1].extract_text()  
            
            #eliminar tildes y convertir todo texto a minusculas.
            texto=strip_accents(texto)
            #texto = limpiar_texto(texto)

            print(f"Texto extraído del archivo {filename}:\n{texto}\n")

            #calcular periodo actual.
            periodo = datetime.now().strftime('%Y-%m-%d %H:%M')
            print(f'periodo actual: {periodo}')

            ## Extraer información comercial ##
            #extraer nic
            nic = texto[:5]
            print(f'nic: {nic}')

            ## Extraer infromación de factura ##
            #extraer periodo de facturación
            periodo_fact = re.search(r'\d{2}/\d{2}/\d{4} a \d{2}/\d{2}/\d{4}\s*(\d+)', texto)
            periodo_fact = periodo_fact.group(0) if periodo_fact else None
            print(periodo_fact)

            #extraer periodo de facturación inicial
            periodo_fact_inicial = periodo_fact[:11]
            print(f'periodo facturado inicial: {periodo_fact_inicial}')

            #extraer periodo de facturación final
            periodo_fact_final = periodo_fact[13:23]
            print(f'periodo facturado final: {periodo_fact_final}')

            #extraer días de facturación
            dias_fact = periodo_fact[24:]
            print(f'dias facturados: {dias_fact}')

Leyendo archivo: 5244937.pdf
Texto extraído del archivo 5244937.pdf:
- cliente -
vatia s.a. e.s.p
nit. 817.001.892-1
grandes contribuyentes res. dian no. 012635 dic 14/2018
autoretenedor de ica cali art. 104-acuerdo mpal. 0321 / 2011
agente retenedor de impuesto sobre las ventas
av. 6n nro. 47n-32 fax:(2) 666 2400informacion factura
factura no.:
periodo de facturacion:
dias facturados:
fecha expedicion:
fecha de vencimiento:
fecha ultimo pago:
valor ultimo pago:
entidad donde realizo el pago:
informacion cliente
nombre:
nit: codigo envio: codigo postal:
direccion suministro:
municipio - dpto:
direccion envio:
municipio- dpto: codigo corp:id interno:
ref. de pago:
resumen cuenta mensual
valor a pagar:
fecha vencimiento:
consumo facturado:
suspension a partir de:
facturas  vencidas:
motivo suspension del servicio o corte:
empresa vigilada por la superintendencia de servicios publicos ssp - registro nuir: 2-19573001-15244937
01/04/2024 a 30/04/2024
30
 01/05/2024 22:04:04
16/05/2024
15/04

In [29]:
patron_direccion = re.compile(r'^(.*)$', re.MULTILINE)
direccion = patron_direccion.findall(texto)
print(direccion)

['- cliente -', 'vatia s.a. e.s.p', 'nit. 817.001.892-1', 'grandes contribuyentes res. dian no. 012635 dic 14/2018', 'autoretenedor de ica cali art. 104-acuerdo mpal. 0321 / 2011', 'agente retenedor de impuesto sobre las ventas', 'av. 6n nro. 47n-32 fax:(2) 666 2400informacion factura', 'factura no.:', 'periodo de facturacion:', 'dias facturados:', 'fecha expedicion:', 'fecha de vencimiento:', 'fecha ultimo pago:', 'valor ultimo pago:', 'entidad donde realizo el pago:', 'informacion cliente', 'nombre:', 'nit: codigo envio: codigo postal:', 'direccion suministro:', 'municipio - dpto:', 'direccion envio:', 'municipio- dpto: codigo corp:id interno:', 'ref. de pago:', 'resumen cuenta mensual', 'valor a pagar:', 'fecha vencimiento:', 'consumo facturado:', 'suspension a partir de:', 'facturas  vencidas:', 'motivo suspension del servicio o corte:', 'empresa vigilada por la superintendencia de servicios publicos ssp - registro nuir: 2-19573001-15244937', '01/04/2024 a 30/04/2024', '30', ' 01/0

## información del cliente

In [38]:
datos_tienda = {}

In [40]:
info_tienda = re.compile(r'jeronimo martins colombia sas tienda\s+([\d.]+)\s+(\b\d{9}\b)')
match = info_tienda.search(texto)
#+^(.*)$
if match:
    datos_tienda['info_tienda'] = {
        'nro_tienda': match.group(1),
        'nit': match.group(2)}
    
# 'nit': match.group(2),
        # 'direccion': match.group(3)
    
print(datos_tienda)

{'info_tienda': {'nro_tienda': '24', 'nit': '900480569'}}


## Información de la factura

In [31]:
datos_factura = {}

In [43]:

patron_activa = re.compile(r'activa kwh\s+([\d.]+)\s+\$\s+([\d.,]+)\s+\$\s+([\d.,]+)')
match_activa = patron_activa.search(texto)

if match_activa:
    datos_factura['activa_kwh'] = {
        'cantidad': match_activa.group(1),
        'valor_energia': match_activa.group(2),
        'valor_contribucion': match_activa.group(3)}

patron_ind_facturada = re.compile(r'reactiva ind facturada kvarh\s+([\d.]+)\s+([\d.]+)\s+\$\s+([\d.,]+)\s+\$\s+([\d.,]+)')
match_ind_facturada = patron_ind_facturada.search(texto)

if match_ind_facturada:
    datos_factura['reactiva_ind_facturada_kvarh'] = {
        'cantidad': match_ind_facturada.group(1),
        'factor': match_ind_facturada.group(2),
        'valor_energia': match_ind_facturada.group(3),
        'valor_contribucion': match_ind_facturada.group(4)
    }

patron_capacitiva = re.compile(r'reactiva capacitiva kvarh\s+([\d.]+)\s+([\d.]+)\s+\$\s+([\d.,]+)\s+\$\s+([\d.,]+)')
match_capacitiva = patron_capacitiva.search(texto)

if match_capacitiva:
    datos_factura['reactiva_capacitiva_kvarh'] = {
        'cantidad': match_capacitiva.group(1),
        'factor': match_capacitiva.group(2),
        'valor_energia': match_capacitiva.group(3),
        'valor_contribucion': match_capacitiva.group(4)
    }
subtotal_energia = re.compile(r'subtotal energia \s+\$\s+([\d.,]+)\s+\$\s+([\d.,]+)')
match_subtotal = subtotal_energia.search(texto)
    
if match_subtotal:
    datos_factura['subtotal_energia'] = {
        'valor_energia': match_subtotal.group(1),
        'valor_contribucion': match_subtotal.group(2)
    }

energia_contribucion = re.compile(r'subtotal energia \s+\$\s+([\d.,]+)')
match_ene_contr= energia_contribucion.search(texto)

if match_ene_contr:
    datos_factura['energia_contribucion'] = {
        'valor_contribucion': match_ene_contr.group(1)
    }

total_otros_cobros = re.compile(r'total otros cobros\s+([\d.,]+)')
match_total_otros_cobros = total_otros_cobros.search(texto)

if match_total_otros_cobros:
    datos_factura['total_otros_cobros'] = {
        'valor_facturado': match_total_otros_cobros.group(1)}

print(datos_factura)

{'activa_kwh': {'cantidad': '28968.43', 'valor_energia': '24,299,573.07', 'valor_contribucion': '4,859,914.36'}, 'reactiva_ind_facturada_kvarh': {'cantidad': '2.13', 'factor': '1', 'valor_energia': '541.24', 'valor_contribucion': '108.25'}, 'reactiva_capacitiva_kvarh': {'cantidad': '.00', 'factor': '1', 'valor_energia': '.00', 'valor_contribucion': '.00'}, 'subtotal_energia': {'valor_energia': '24,300,114.31', 'valor_contribucion': '4,860,022.60'}, 'energia_contribucion': {'valor_contribucion': '24,300,114.31'}, 'total_otros_cobros': {'valor_facturado': '3,605,723.00'}}


In [45]:
df_factura = pd.DataFrame(datos_factura)

# Transponer el DataFrame para que las claves sean las filas
#df_factura = df_factura.transpose()

# Mostrar el DataFrame
print(df_factura)

                       activa_kwh reactiva_ind_facturada_kvarh  \
cantidad                 28968.43                         2.13   
valor_energia       24,299,573.07                       541.24   
valor_contribucion   4,859,914.36                       108.25   
factor                        NaN                            1   
valor_facturado               NaN                          NaN   

                   reactiva_capacitiva_kvarh subtotal_energia  \
cantidad                                 .00              NaN   
valor_energia                            .00    24,300,114.31   
valor_contribucion                       .00     4,860,022.60   
factor                                     1              NaN   
valor_facturado                          NaN              NaN   

                   energia_contribucion total_otros_cobros  
cantidad                            NaN                NaN  
valor_energia                       NaN                NaN  
valor_contribucion        24,

## Datos cobros

In [33]:
datos_otros_cobros = {}

In [34]:

#Saldo cartera
saldo_cartera = re.compile(r'saldo cartera\s+\$+([\d.,]+)\s+\$+([\d.,]+)')
match_saldo = saldo_cartera.search(texto)

if match_saldo:
    datos_otros_cobros['saldo_cartera'] = {
        'Saldo financiado': match_saldo.group(1),
        'valor_facturado': match_saldo.group(2)}

plan_datos = re.compile(r'plan de datos\s+\$+([\d.,]+)\s+\$+([\d.,]+)')
match_plan_datos = plan_datos.search(texto)

if match_plan_datos:
    datos_otros_cobros['plan_datos'] = {
        'Saldo financiado': match_plan_datos.group(1),
        'valor_facturado': match_plan_datos.group(2)}    

alum_publico = re.compile(r'alumbrado pub vlr \s+\$+([\d.,]+)\s+\$+([\d.,]+)')
match_alum_publico = alum_publico.search(texto)

if match_alum_publico:
    datos_otros_cobros['alum_publico'] = {
        'Saldo financiado': match_alum_publico.group(1),
        'valor_facturado': match_alum_publico.group(2)}  
    
valor_iva = re.compile(r'valor iva\s+\$+([\d.,]+)\s+\$+([\d.,]+)')
match_valor_iva = valor_iva.search(texto)

if match_valor_iva:
    datos_otros_cobros['valor_iva'] = {
        'Saldo financiado': match_valor_iva.group(1),
        'valor_facturado': match_valor_iva.group(2)} 
    
imp_seg_valor = re.compile(r'imp seguridad vlr\s+\$+([\d.,]+)\s+\$+([\d.,]+)')
match_imp_seg_valor = imp_seg_valor.search(texto)

if match_imp_seg_valor:
    datos_otros_cobros['imp_seg_valor'] = {
        'Saldo financiado': match_imp_seg_valor.group(1),
        'valor_facturado': match_imp_seg_valor.group(2)} 
    
# total_otros_cobros = re.compile(r'total otros cobros\s+([\d.,]+)')
# match_total_otros_cobros = total_otros_cobros.search(texto)

# # if match_total_otros_cobros:
# #     total_otros_cobros['total_otros_cobros'] = {
# #         'valor_facturado': match_total_otros_cobros.group(1)}
    
print(datos_otros_cobros)

{'saldo_cartera': {'Saldo financiado': '.00', 'valor_facturado': '.00'}, 'plan_datos': {'Saldo financiado': '.00', 'valor_facturado': '7,241.00'}, 'valor_iva': {'Saldo financiado': '.00', 'valor_facturado': '1,376.00'}, 'imp_seg_valor': {'Saldo financiado': '.00', 'valor_facturado': '413,100.31'}}


In [37]:
df = pd.DataFrame(datos_otros_cobros)

# Transponer el DataFrame para que las claves sean las filas
#df = df.transpose()

# Mostrar el DataFrame
print(df)

                 saldo_cartera plan_datos valor_iva imp_seg_valor
Saldo financiado           .00        .00       .00           .00
valor_facturado            .00   7,241.00  1,376.00    413,100.31


In [ ]:
df_pivot = df.pivot

# Transponer el DataFrame para que las claves sean las filas
df = df.transpose()

# Mostrar el DataFrame
print(df)